Problema:

Propor um classificador que identifique o grupo de cada objeto.

Dados:

* $g$: número de grupos diferentes
* $n$: número de objetos (não necessariamente diferentes)
* $n_{min}$: número mínimo de objetos em um grupo
* $n_{max}$: número máximo de objetos em um grupo

Para cada Objeto:

* $c$: número de características binárias
* $c_y$: número de características de um determinado grupo
* $c_n$: número de características dos demais grupos ($c_n = c_y (g - 1)$)
* $p$: probabilidade de ativação das características de um grupo ($p > 0.5$)
* $1 - p$: probabilidade de ativação das características dos demais grupos
* $p' = 0.5$: probabilidade de ativação das características que não são de qualquer grupo
* (as características de cada grupo não tem interseção)

## Gerador de Instância

In [2]:
"gera a distribuição de objetos para os grupos"
function group_size(g, n, n_min, n_max)
    num_g = Array(Int, g)
    sum = 0
    for i=1:g
        num_g[i] = rand(n_min:n_max)
        sum += num_g[i]
    end
    correct = n / sum
    sum = 0
    for i=1:g
        num_g[i] = round(Int, num_g[i] * correct)
        sum += num_g[i]
    end
    if sum < n
        num_g[g] += 1
    end
    num_g
end

"máscara de características para cada grupo sem interseção"
function group_mask(g, c, c_y)
    char_g = fill(-1, c)
    index = 1
    for i=1:g, j=1:c_y
        char_g[index] = i
        index += 1
    end
    char_g
end

"""gera objetos para grupos seguindo a distribuição num_g,
a máscara char_g e a probabilidade p de ativação"""
function generate_data(num_g, char_g, p)
    data = Array(Tuple{Array{Int,1},Int}, 0)
    for i=1:length(num_g),j=1:num_g[i]
        vect = zeros(Int, length(char_g))
        for k=1:length(vect)
            if char_g[k] == i
                vect[k] = rand() < p ? 1 : 0
            elseif char_g[k] != -1
                vect[k] = rand() < 1 - p ? 1 : 0
            else
                vect[k] = rand() < 0.5 ? 1 : 0
            end
        end
        push!(data, (vect, i))
    end
    data
end

"gerador de instâncias para o problema de clusterização"
function oc152_t_instance_generator(n, c, c_y, p, g, n_min, n_max)
    if c < g * c_y
        println("c_y too big")
        return nothing
    end
    
    num_g = group_size(g, n, n_min, n_max)
    char_g = group_mask(g, c, c_y)
    data = generate_data(num_g, char_g, p)
    data
end

oc152_t_instance_generator (generic function with 1 method)

In [3]:
function data(;n = 20, n_min = 0, n_max = 0, g = 5, c = 16, c_y = 3, p = 0.8)
    if n < 10
        println("minimum 10")
        return nothing
    end
    if g > n
        println("too many groups")
        return nothing
    end
    if c < g * c_y
        println("c_y too big")
        return nothing
    end
    
    if n_max == 0
        n_max = 2 * round(Int, n / g)
    end
    if n_min == 0
        n_min = round(Int, n_max / 10)
    end
    if n_max * g < n
        println("n_max too tight")
        return nothing
    end
    
    data = oc152_t_instance_generator(n, c, c_y, p, g, n_min, n_max)
    shuffle!(data)
end

data (generic function with 1 method)

### ULP - Problema de Localização sem Capacidade

Consiste em resolver o *ULP* determinar os objetos representates de cada grupo e classificar cada objeto como sendo do grupo com representante *mais próximo*